In [9]:
import sys, os, os.path
import glob

import scipy as sp
import numpy as np

import matplotlib
import matplotlib.pyplot as pp

import yt
from yt.frontends.boxlib.data_structures import AMReXDataset
yt.funcs.mylog.setLevel(50)

In [10]:
%matplotlib inline

In [11]:
sys.path.append("src_analysis")
from data_model import *
from visualize  import *

In [12]:
sys.path.append("chlamy-flagellum-master")
import pychlamy.data_model       as cdm
from   pychlamy.geometry         import DiscreteCurve
from   pychlamy.fourier_analysis import FourierTransform
from   scipy.io                  import loadmat

In [13]:
def get_line(lines, name):
    """
    Select line based on variable name, ignoring comments
    """
    for line in lines:
        data = line.strip()
        if data.startswith(name):
            return data
    return None

def select_data(x):
    """
    Select the substring after the '=' sign
    """
    return x.split("=")[1].strip()

def select_vector(x):
    """
    Select the last 3 (space-delimited) non-empty substrings after the '=' sign
    """
    data = select_data(x)
    return list(filter(lambda x:len(x)>0, data.split(" ")))[:3]

In [14]:
def get_timings(inputs_file_path):
    with open(inputs_file_path) as f:
        lines = f.readlines()

    fixed_dt  = float(select_data(get_line(lines, "fixed_dt")))
    plot_int  = int(select_data(get_line(lines, "plot_int")))
    frequency = float(select_data(get_line(lines, "frequency")))

    return fixed_dt*plot_int, frequency

In [15]:
def get_dimension(inputs_file_path):
    with open(inputs_file_path) as f:
        lines = f.readlines()

    lo = map(lambda x:float(x), select_vector(get_line(lines, "prob_lo")))
    hi = map(lambda x:float(x), select_vector(get_line(lines, "prob_hi")))
    
    return np.array(list(lo)), np.array(list(hi))

def get_property(inputs_file_path, prop, parser):
    with open(inputs_file_path) as f:
        lines = f.readlines()

    return parser(select_data(get_line(lines, prop)))

In [16]:
def shift_pbc(pos, prev_pos, size):
    new_pos = np.zeros(3)
    for i in range(3):
        new_pos[i] = pos[i]
        if (pos[i] - prev_pos[i]) > size[i]/2:
            new_pos[i] = pos[i] - size[i]
        if (pos[i] - prev_pos[i]) < -size[i]/2:
            new_pos[i] = pos[i] + size[i]
    return new_pos

In [17]:
def avg_pos(data_file, prev_pos, size):
    ds = yt.load(data_file)

    amrex_ds = AMReXDataset(data_file)
    ad = amrex_ds.all_data()
    aos = AoS(ad)

    p_x = 0
    p_y = 0
    p_z = 0
    n = 0

    for j, part in enumerate(aos.particles):
        p = shift_pbc(part.pos, prev_pos, size)
        p_x += p[0]
        p_y += p[1]
        p_z += p[2]
        n += 1

    return np.array([p_x/n, p_y/n, p_z/n])

In [18]:
# data_root = "/home/shared/gxu"
data_path = "/home/mhuber/Project/FHDeX/exec/immersed_boundary/taylor_line"
inputs_file = os.path.join(data_path, "inputs_taylor")

In [19]:
n_fill   = 7
prefix   = "plt"

In [20]:
data_glob  = os.path.join(data_path, prefix + "*")
data_files = glob.glob(data_glob)
data_files.sort()

In [21]:
data_files

['/home/mhuber/Project/FHDeX/exec/immersed_boundary/taylor_line/plt0000000',
 '/home/mhuber/Project/FHDeX/exec/immersed_boundary/taylor_line/plt0000200',
 '/home/mhuber/Project/FHDeX/exec/immersed_boundary/taylor_line/plt0000400',
 '/home/mhuber/Project/FHDeX/exec/immersed_boundary/taylor_line/plt0000600',
 '/home/mhuber/Project/FHDeX/exec/immersed_boundary/taylor_line/plt0000800',
 '/home/mhuber/Project/FHDeX/exec/immersed_boundary/taylor_line/plt0001000',
 '/home/mhuber/Project/FHDeX/exec/immersed_boundary/taylor_line/plt0001200',
 '/home/mhuber/Project/FHDeX/exec/immersed_boundary/taylor_line/plt0001400',
 '/home/mhuber/Project/FHDeX/exec/immersed_boundary/taylor_line/plt0001600',
 '/home/mhuber/Project/FHDeX/exec/immersed_boundary/taylor_line/plt0001800',
 '/home/mhuber/Project/FHDeX/exec/immersed_boundary/taylor_line/plt0002000',
 '/home/mhuber/Project/FHDeX/exec/immersed_boundary/taylor_line/plt0002200',
 '/home/mhuber/Project/FHDeX/exec/immersed_boundary/taylor_line/plt0002400',

In [22]:
ds.field_list

NameError: name 'ds' is not defined

In [25]:
# don't use `plot_index = -1` here because this would get the equilibrium 
# configuration's time wrong
# plot_index = len(data_files) - 1
plot_index = 4

# set up plot
#prop = PlotProperies(marker_rad=2e-5, slc_name="pres", slc_max=1e-2)
prop = PlotProperies(marker_rad=2e-5, slc_name=[("boxlib", "avg_velx"), ("boxlib", "avg_vely"), ("boxlib", "avg_velz"), ("boxlib", "pres")], slc_max=1e-2)
# load data
ds = yt.load(data_files[plot_index])
# plot everything
slc  = slice_plt(ds, prop)
add_ibm(slc, data_files[plot_index], prop)
slc.annotate_quiver("cc_forcex", "cc_forcey", plot_args={"color": "red"}, scale=1e5)
slc.show()

/home/mhuber/miniconda3/lib/python3.10/site-packages/yt/visualization/plot_modifications.py:755: VisibleDeprecationWarning: `plot_args` is deprecated. You can now pass arbitrary keyword arguments instead of a dictionary.
Deprecated since yt 4.1.0
This feature is planned for removal two minor releases later (anticipated)
  super().__init__(


YTPlotCallbackError: annotate_sphere callback failed

In [23]:
for i, data_file in enumerate(data_files):
    frame_file = "pictureRun47two" + str(i).zfill(5) + ".png"

    if os.path.isfile(frame_file):
        continue
    else:
        print(f"Working on: {frame_file}", end="\r")

    # load data
    ds = yt.load(data_file)

    # plot everything
    slc  = slice_plt(ds, prop)
    add_ibm(slc, data_file, prop)
    # add_equilibrium(slc, dc, theta_nodes, data_files, i, dt, plot_int, frequency, prop)
    slc.annotate_quiver("cc_forcex", "cc_forcey", plot_args={"color": "red"}, scale=1e6)

    slc.save(frame_file)

NameError: name 'prop' is not defined

In [24]:
#from IPython.display import Image
#Image(filename='picture600400.png')